# Advent of Code 2020 - Day 8

### Challenge

--- Day 8: Handheld Halting ---
Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
These instructions are visited in this order:

nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

-------------

In [136]:
from aocd import data

In [137]:
data = list(data.split('\n'))

In [138]:

data2 = list(x.split() for x in data)

In [139]:
data = data2

In [140]:
data[0:5]

[['jmp', '+336'],
 ['jmp', '+593'],
 ['jmp', '+121'],
 ['acc', '-8'],
 ['nop', '+459']]

In [91]:
def run_code(data):
    pointer = 0
    accumulator = 0
    visited = []

    while pointer not in visited:
        visited.append(pointer)

        if data[pointer][0] == 'acc':
            accumulator += int(data[pointer][1])
            pointer += 1

        elif data[pointer][0] == 'jmp':
            pointer += int(data[pointer][1])

        elif data[pointer][0] == 'nop':
            pointer += 1

    return accumulator


In [90]:
def run_code_test(data):
    pointer = 0
    accumulator = 0
    visited = []

    try:
        while pointer not in visited:
            visited.append(pointer)

            if data[pointer][0] == 'acc':
                accumulator += int(data[pointer][1])
                pointer += 1

            elif data[pointer][0] == 'jmp':
                pointer += int(data[pointer][1])

            elif data[pointer][0] == 'nop':
                pointer += 1

        return accumulator
    except:
        print("error on location", pointer, ". Accumulator currently at: ", accumulator)

In [16]:
accumulator

1766

--------

## Part 2

--- Part Two ---
After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

-----------

In [20]:
data[627]

['jmp', '+1']

entry 627 is the last line. 

we need an instruction chain that gets the pointer to 627

Let's first try if there currently is a jump that achieves that, or a NOP that can be turned into a jmp to achieve this. 

In [31]:
for i in range(627):
    if data[i][0] in ['jmp', 'nop']:
        if i + int(data[i][1]) == 627:
            print(i, data[i])
    

626 ['jmp', '+1']


626 gets us to 627, but is there anything that gets us to 626?

In [32]:
for i in range(627):
    if data[i][0] in ['jmp', 'nop']:
        if i + int(data[i][1]) == 626:
            print(i, data[i])
    

No results, there don't seem to be any jmp or nop that can directly get us to 626 and 627, we will need to include the acc

from the top of the code, we are going to turn nop into jmp and vice versa, one at a time, and run the code in between.

We want to generate a 'index out of range' error, as that likely suggests we've reached the last instruction.

Once we've found the location we need to change, we can swap that instruction and record the accumulator

In [73]:
searching = True
i = 0
while(searching):
    for j in range(i, len(data)):
        if data[j][0] == 'nop':
            data[j][0] = 'jmp'
            try:
                run_code(data)
                i = j+1
                break
            except:
                print("Error after changing location", i+j )
                searching = False
                break
            
        elif data[j][0] == 'jmp':
            data[j][0] = 'nop'
            try:
                run_code(data)
                i = j+1
                break
            except:
                print("Error after changing location", i+j )
                searching = False
                break
        

Error after changing location 204


Let's make a copy of our dataset that we can edit without affecting the original. 

We need a deep copy here as we are editing individual elements in the list, so all elements need to be duplicated and newly referenced in the new list

In [114]:
import copy

In [142]:
data_copy = copy.deepcopy(data)

In [143]:
data_copy[204][0] = 'nop'

In [145]:
data_copy[204]

['nop', '+19']

In [89]:
run_code_test(data3)

error on location 628 . Accumulator currently at:  1639
